In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [3]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [4]:
from core.agent import *
from core.world import * 
from core.render import * 
from core.skill import * 
from core.models import *
from core.message import *

In [5]:
DEVICE = "cuda"

In [6]:
from sar.sar_agent import *
from sar.sar_world import *
from sar.sar_env_params import *
from sar.sar_traits_sampler import *

belief_initializer = SARBeliefInitializer(BELIEF_DIMS)
trait_sampler = SARTraitSampler()

def initialize_swarm(world : BaseWorld):
    swarm = trait_sampler.generate(SWARM_SIZE, DEVICE)
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    swarm = belief_initializer.initialize_beliefs(swarm)

In [ ]:

from sar.urban_gen import * 
from sar.victims import * 
from sar.sar_comm import *
from utils.geo import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)

GEO_TIFF_PATH = "C:\\Users\\Joaquin\\Desktop\\Thesis-Playground\\datasets\\output_SRTMGL1.tif"

def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    geo_map = read_tiff(GEO_TIFF_PATH)[0]
    tx, ty = geo_map.shape

    if tx > world._dims[0] or ty > world._dims[1]:
        start_x = np.random.randint(0, tx - world._dims[0] + 1)
        start_y = np.random.randint(0, ty - world._dims[1] + 1)
        height_map = geo_map[start_x:start_x + world._dims[0], start_y:start_y + world._dims[1]]
        minimum = height_map.min()
        maximum = height_map.max() 
        terrain_map = TerrainMap(height_map, terrain_map._padding, minimum, maximum)


    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [8]:
from sar.energy import EnergyModel
from sar.victims import VictimModel
from solution.sar_action_interpreter import *
from solution.encoder_net import *
from solution.decoder_net import *
from models.complex_model import * 

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

0
-2


In [9]:
grid_size = 2 * 3 + 1
policy_net= PolicyNet(1, grid_size, 4)
target_net=  PolicyNet(1, grid_size, 4)
encoder_model = Encoder(hidden_dim = 32, output_dim= 16)
decoder_model = Decoder(input_dims= 16, hidden_dim = 16)

complex_model = ComplexModel(
    policy_net= policy_net, 
    encoder_net = encoder_model, 
    decoder_net = decoder_model
)

comms_protocol = SARCommunicationProtocol(encoder_model, decoder_model)
action_interpreter = SARActionInterpreter(BELIEF_DIMS)

custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter, comms_protocol)

complex_model.to(DEVICE)
custom_gym.to(DEVICE)

In [10]:
custom_gym.reset(42)

37


({1: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 1., 0.],
          [0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 1.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 0.]]),
   'State': tensor([114.4653,   1.0000,   1.0000], device='cuda:0')},
  2: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 0., 0., 0., 0.],
          [1., 0., 0., 1., 1.],
          [1., 0., 0., 0., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 0., 1., 0.]]),
   'State': tensor([101.9043,   1.0000,   1.0000], device='cuda:0')},
  3: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]]),
   'State': tensor([125.4518,   0.0000,   0.0000], device='cuda:0')},
  4: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 0., 0., 1., 1.],
    

### Testing

In [11]:
parallel_api_test(custom_gym, num_cycles=1_000)
custom_gym.reset()

-7
37
37
Passed Parallel API test
37


({1: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 1., 0.],
          [0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 1.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 0.]]),
   'State': tensor([114.4653,   1.0000,   1.0000], device='cuda:0')},
  2: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 0., 0., 0., 0.],
          [1., 0., 0., 1., 1.],
          [1., 0., 0., 0., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 0., 1., 0.]]),
   'State': tensor([101.9043,   1.0000,   1.0000], device='cuda:0')},
  3: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]]),
   'State': tensor([125.4518,   0.0000,   0.0000], device='cuda:0')},
  4: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 0., 0., 1., 1.],
    

# Training

In [12]:

from models.base import * 
from models.idqn import * 
from solution.policy_net import PolicyNet
import matplotlib.pyplot as plt

In [13]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             target_net= target_net,
             model= complex_model,
             batch_size=1024,
             device = DEVICE,
             buffer_size= custom_gym._max_time_steps * 10
             )

In [ ]:
rewards = train_loop(custom_gym, model, games=100, optimization_passes = 1, seed=42)

Training on thesis.
37


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

37
37


In [ ]:
plt.plot(rewards)

# Optimizations

In [ ]:
import cProfile

In [ ]:
def stress_test():
    train_loop(custom_gym, model, games=1, optimization_passes = 1, seed=42)

In [ ]:

cProfile.run('stress_test()', sort = 'time')

# Visualization

In [ ]:
from models.base import RenderWrapper

renderer = RenderWrapper(model=model)

In [ ]:
renderer.render()